# Proceso Pansharpening google Earth Engine

In [1]:
# Importar GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee,{"color" : "00FF11"},name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccionar Collection Landsat TOA

In [6]:
# Buscar imagenes Landsat por zona estudio shapefile y nubosidad
L8_TOA = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")\
                .filterDate('2020-01-01','2020-12-31')\
                .filterBounds(zona_ee)\
                .filterMetadata('CLOUD_COVER','less_than',20)

In [7]:
# Conteo de imagenes
count = L8_TOA.size().getInfo()
print("Cantidad de imagenes L8_TOA:", count)

Cantidad de imagenes L8_TOA: 3


In [8]:
# Imprimir la lista de ID Imagenes
catalogo_L8_TOA = L8_TOA.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(catalogo_L8_TOA)

['LC08_003069_20200420', 'LC08_003069_20200607', 'LC08_003069_20200911']


In [9]:
catalogo_L8_TOA[0]

'LC08_003069_20200420'

## Seleccion imagen

In [10]:
# Importar imagen Landsat 8 TOA
image = ee.Image("LANDSAT/LC08/C01/T1_TOA" + "/" + catalogo_L8_TOA[0])

In [11]:
# Nombre de las bandas
image.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [12]:
# Simbologia estala 0 - 1
viz_TOA = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 0,
    'max': 0.5,
    'gamma': 1.5
}

## Recortar imagen

In [13]:
## Recortar Imagen
image_clip = image.clip(zona_ee)

In [14]:
Map.addLayer(image_clip, viz_TOA, name = "Landsat B654")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

In [15]:
# Seleccion combinacion bandas
veg = ['B6','B5','B4']
Nat = ['B4','B3','B2']

## Convertir RGB a HSV

In [16]:
# Convertir de RGB a HSV
RGB2HSV_veg = image_clip.select(veg).rgbToHsv()
RGB2HSV_nat = image_clip.select(Nat).rgbToHsv()

In [18]:
# Nombre de las bandas
RGB2HSV_nat.bandNames().getInfo()

['hue', 'saturation', 'value']

## Agregar banda 8 y convertir RGB

In [19]:
# Agregar banda 8 y convertir RGB
Pansharpen_veg = ee.Image.cat(
                RGB2HSV_veg.select("hue"),
                RGB2HSV_veg.select("saturation"),
                image_clip.select("B8")).hsvToRgb()

Pansharpen_nat = ee.Image.cat(
                RGB2HSV_nat.select("hue"),
                RGB2HSV_nat.select("saturation"),
                image_clip.select("B8")).hsvToRgb()

In [21]:
# Determinar el nombre de la banda
Pansharpen_nat.bandNames().getInfo()

['red', 'green', 'blue']

In [22]:
# simbologia landsat 8 Pansharpening Escala 0 - 1
viz_pan = {
    'bands': ['red','green','blue'],
    'min': 0,
    'max': 0.5,
    'gamma': 1.5
}

In [28]:
# Reproyeccion del pansharpening
Pansharpen_nat_UTM = Pansharpen_nat.reproject(crs="EPSG:32719", scale = 15)
Pansharpen_veg_UTM = Pansharpen_veg.reproject(crs="EPSG:32719", scale = 15)

In [23]:
Map.addLayer(Pansharpen_nat_UTM, viz_pan, name = "Pansharpening Natural B432")
Map.addLayer(Pansharpen_veg_UTM, viz_pan, name = "Pansharpening Vegetacion B654")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Crear una visualización comparación

In [24]:
# Crear visualizacion derecha y izquierda
viz_izq = geemap.ee_tile_layer(image_clip, viz_TOA, 'Landsat B654')
viz_der = geemap.ee_tile_layer(Pansharpen_veg_UTM, viz_pan, 'Pansharpening Vegetacion B654')

In [25]:
# Visualizacion
Map.split_map(viz_izq, viz_der)
Map

Map(bottom=1125100.0, center=[-12.972442010578366, -70.17062187194826], controls=(ZoomControl(options=['positi…

## Exportar Imagen

In [26]:
# Exportar en google drive
geemap.ee_export_image_to_drive(Pansharpen_veg_UTM, description='Pansharpen_clip_B654', folder='GEE_Python', region=geometria, scale=15)

Exporting Pansharpen_clip_B654 ...


In [ ]:
geemap.ee_export_image_to_drive(Pansharpen_nat_UTM, description='Pansharpen_clip_B432', folder='GEE_Python', region=geometria, scale=15)